In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Projects/Kaggle/02. House Price Prediction/01. DataSet/train.csv')
print('Shape:',df.shape)
df.sample(5)

Shape: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
557,558,50,C (all),60.0,11040,Pave,NaN,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1.5Fin,4,6,1920,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,637,Unf,0,0,637,GasA,Gd,Y,SBrkr,897,439,0,1336,0,0,1,1,3,1,TA,7,Typ,0,NaN,CarPort,1994.0,Unf,1,570,TA,TA,Y,0,47,120,0,0,0,NaN,NaN,NaN,0,9,2006,COD,Normal,108000
718,719,60,RL,96.0,10542,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,2Story,7,5,1993,1994,Hip,CompShg,Wd Sdng,ImStucc,BrkFace,651.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,1173,Unf,0,138,1311,GasA,Ex,Y,SBrkr,1325,1093,0,2418,1,0,2,1,3,1,Gd,9,Typ,1,TA,Attchd,1993.0,RFn,3,983,TA,TA,Y,250,154,216,0,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal,341000
1412,1413,90,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,Duplex,1Story,4,5,1949,1950,Gable,CompShg,BrkFace,Stone,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,Wall,Fa,N,FuseF,1040,0,0,1040,0,0,2,0,2,2,TA,6,Typ,0,NaN,Detchd,1956.0,Unf,2,420,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,90000
1370,1371,50,RL,90.0,5400,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,4,6,1920,1950,Gable,CompShg,CBlock,CBlock,None,0.0,Fa,TA,PConc,TA,TA,No,ALQ,315,Rec,105,420,840,GasA,Ex,Y,SBrkr,840,534,0,1374,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1967.0,Fin,1,338,TA,TA,Y,0,0,198,0,0,0,NaN,NaN,NaN,0,10,2009,WD,Normal,105000
974,975,70,RL,60.0,11414,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,BrkSide,RRAn,Feedr,1Fam,2Story,7,8,1910,1993,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,Gd,BrkTil,Gd,TA,No,Unf,0,Unf,0,728,728,GasA,TA,N,SBrkr,1136,883,0,2019,0,0,1,0,3,1,Gd,8,Typ,0,NaN,Detchd,1997.0,Unf,2,532,TA,TA,Y,509,135,0,0,0,0,NaN,GdPrv,NaN,0,10,2009,WD,Normal,167500


In [ ]:
#df.info()

In [4]:
#Categorizing columns into Numerical or Categorical (norminal/ordinal) data types 

numerical_columns= ['Id', 'LotFrontage', 'LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','Bedroom','Kitchen','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']
categorical_nominal_columns= ['MSSubClass','LotShape', 'OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','CentralAir','KitchenQual','FireplaceQu','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence']
categorical_ordinal_columns= ['MSZoning', 'Street', 'Alley','LandContour', 'Utilities', 'LotConfig','LandSlope','Neighborhood', 'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','Electrical','Functional','GarageType','MiscFeature','SaleType','SaleCondition']
#Some of ordinal columns could be nominal which can only be identified by checking comparing it with the output

output_column= ['SalePrice']

## 1. Converting categorical nominal columns to numerical columns by label encoding

This will help in finding the correlation to the output and ease the feature selection process

In [5]:
def LotShape(val):
  lables={'Reg':3,'IR1':2,'IR2':3,'IR3':4}
  if val in lables: return lables[val]
  else:return val

def Multiple(val):
  lables={'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}
  if val in lables: return lables[val]
  else:return val

def BsmtExposure(val):
  lables={'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0}
  if val in lables: return lables[val]
  else:return val

def BsmtFinType(val):
  lables={'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}
  if val in lables: return lables[val]
  else:return val

def CentralAir(val):
  lables={'Y':1,'N':0}
  if val in lables: return lables[val]
  else:return val

def GarageFinish(val):
  lables={'Fin':3,'RFn':2,'Unf':1,'NA':0}
  if val in lables: return lables[val]
  else:return val

def PavedDrive(val):
  lables={'Y':2,'P':1,'N':0}
  if val in lables: return lables[val]
  else:return val

def Fence(val):
  lables={'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'NA':0}
  if val in lables: return lables[val]
  else:return val

df['LotShape']=df['LotShape'].apply(LotShape)
df['BsmtExposure']=df['BsmtExposure'].apply(BsmtExposure)
df['BsmtFinType1']=df['BsmtFinType1'].apply(BsmtFinType)
df['BsmtFinType2']=df['BsmtFinType2'].apply(BsmtFinType)
df['CentralAir']=df['CentralAir'].apply(CentralAir)
df['GarageFinish']=df['GarageFinish'].apply(GarageFinish)
df['PavedDrive']=df['PavedDrive'].apply(PavedDrive)
df['Fence']=df['Fence'].apply(Fence)
df['ExterQual']=df['ExterQual'].apply(Multiple)
df['ExterCond']=df['ExterCond'].apply(Multiple)
df['BsmtQual']=df['BsmtQual'].apply(Multiple)
df['BsmtCond']=df['BsmtCond'].apply(Multiple)
df['HeatingQC']=df['HeatingQC'].apply(Multiple)
df['KitchenQual']=df['KitchenQual'].apply(Multiple)
df['FireplaceQu']=df['FireplaceQu'].apply(Multiple)
df['GarageQual']=df['GarageQual'].apply(Multiple)
df['GarageCond']=df['GarageCond'].apply(Multiple)
df['PoolQC']=df['PoolQC'].apply(Multiple)

In [6]:
for i in categorical_nominal_columns:
  print(i)
  print(df[i].unique())

MSSubClass
[ 60  20  70  50 190  45  90 120  30  85  80 160  75 180  40]
LotShape
[3 2 4]
OverallQual
[ 7  6  8  5  9  4 10  3  1  2]
OverallCond
[5 8 6 7 4 2 3 9 1]
ExterQual
[4 3 5 2]
ExterCond
[3 4 2 1 5]
BsmtQual
[ 4.  3.  5. nan  2.]
BsmtCond
[ 3.  4. nan  2.  1.]
BsmtExposure
[ 1.  4.  2.  3. nan]
BsmtFinType1
[ 6.  5.  1.  3.  4. nan  2.]
BsmtFinType2
[ 1.  4. nan  5.  3.  2.  6.]
HeatingQC
[5 4 3 2 1]
CentralAir
[1 0]
KitchenQual
[4 3 5 2]
FireplaceQu
[nan  3.  4.  2.  5.  1.]
GarageFinish
[ 2.  1.  3. nan]
GarageQual
[ 3.  2.  4. nan  5.  1.]
GarageCond
[ 3.  2. nan  4.  1.  5.]
PavedDrive
[2 0 1]
PoolQC
[nan  5.  2.  4.]
Fence
[nan  3.  2.  4.  1.]


## 2. Handling Nulls

In [7]:
check=df.isnull().sum()
column_names=check.index
null_count=check.values
null_columns=[]

for i in range (0,len(null_count)):
  if(null_count[i]!=0):
    null_columns.append(column_names[i])

null_columns

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [8]:
df[null_columns].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

How to handle nulls?
1. Houses with no basement have nulls in basement related columns. So, replace them with 0.
2. Houses with no garage have nulls in basement related columns. So, replace them with 'NoGrg' oo 0. Filling GarageYrBlt with the column's mode: 2005.
3. Houses with no Masonry veneer have nulls in Masonry veneer related columns. So, replace them with 'None' oo 0.
4. Replacing Lotfootage with it's mode
5. Almost 80% of Alley, FireplaceQu, PoolQC, Fence and MuscFeature columns are empty and rows with non-null values does not show significant output. Hence, it's safe to drop this column.
6. 91% of electrical column is SBrkr and hence replacing the 1 null row with the same

In [9]:
# Dropping columns with high # nulls
drop_coulmns=['Alley','PoolQC','Fence','MiscFeature','FireplaceQu']
df.drop(drop_coulmns,axis=1,inplace=True)
null_columns=set(null_columns)-set(drop_coulmns)

In [10]:
#Replacing Nulls 
null_fillers={'BsmtQual':0,'BsmtCond':0,'BsmtExposure':0,'BsmtFinType1':0,'BsmtFinType2':0,'GarageType':'NoGrg','GarageYrBlt':2005,'GarageFinish':0,'GarageQual':0,'GarageCond':0,'LotFrontage':60,'MasVnrType':'None','MasVnrArea':0,'Electrical':'SBrkr'}

for i in null_columns:
  df[i].fillna(null_fillers[i],inplace=True)

check=df.isnull().any()
null_check=check.values
null_check

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [11]:
df.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
389,390,60,RL,96.0,12474,Pave,3,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,10,5,2007,2008,Gable,CompShg,VinylSd,VinylSd,Stone,272.0,5,3,PConc,5.0,3.0,3.0,6.0,1280,1.0,0,402,1682,GasA,5,1,SBrkr,1742,590,0,2332,1,0,2,1,3,1,5,9,Typ,1,BuiltIn,2008.0,3.0,3,846,3.0,3.0,2,196,134,0,0,0,0,0,8,2008,New,Partial,426000
349,350,60,RL,56.0,20431,Pave,3,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,CemntBd,CmentBd,BrkFace,870.0,5,3,PConc,5.0,3.0,1.0,6.0,1410,1.0,0,438,1848,GasA,5,1,SBrkr,1848,880,0,2728,1,0,2,1,4,1,5,10,Typ,2,Attchd,2006.0,3.0,3,706,3.0,3.0,2,0,0,0,0,0,0,0,4,2006,New,Partial,437154
520,521,190,RL,60.0,10800,Pave,3,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,4,7,1900,2000,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,BrkTil,0.0,0.0,0.0,0.0,0,0.0,0,0,0,GasA,3,0,FuseA,694,600,0,1294,0,0,2,0,3,2,3,7,Typ,0,NoGrg,2005.0,0.0,0,0,0.0,0.0,0,220,114,210,0,0,0,0,8,2008,WD,Normal,106250
1152,1153,20,RL,90.0,14115,Pave,2,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,6,7,1956,2004,Gable,CompShg,Stone,Stone,None,0.0,3,3,PConc,3.0,3.0,1.0,5.0,296,6.0,547,230,1073,GasA,5,1,SBrkr,1811,0,0,1811,0,0,1,0,2,1,5,6,Typ,1,Attchd,1956.0,3.0,2,470,3.0,3.0,2,0,0,280,0,0,0,0,7,2006,WD,Abnorml,230000
589,590,40,RM,50.0,9100,Pave,3,Lvl,AllPub,Inside,Gtl,BrkSide,RRAn,Feedr,1Fam,1Story,5,6,1930,1960,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,BrkTil,3.0,3.0,1.0,1.0,0,1.0,0,742,742,GasA,3,1,FuseA,779,0,156,935,0,0,1,0,2,1,3,4,Typ,0,Detchd,1988.0,1.0,1,308,3.0,3.0,1,0,0,0,0,0,0,600,8,2008,WD,Normal,79500


### 3. Converting date columns to numberical columns & seperating output column

Date columns that needed to be handled:
1. YearBuilt - Subtract it with YrSold and use it as a numerical column
2. YearRemodAdd - Subtract it with YrSold and use it as a numerical column
3. GarageYrBlt- Subtract it with YrSold and use it as a numerical column
4. MoSold - not needed after executing 1,2 & 3 
5. YrSold - not needed after executing 1,2 & 3



In [12]:
df['House_age_from_built']=df['YrSold']-df['YearBuilt']
df['House_age_from_Modified']=df['YrSold']-df['YearRemodAdd']
df.drop(['YearBuilt','YrSold','YearRemodAdd'],axis=1, inplace=True)

#Sepearating 'SalePrice' as the output column
df_op = pd.DataFrame(df.pop('SalePrice'))
df_ip=df.copy()

df_ip.sample(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,SaleType,SaleCondition,House_age_from_built,House_age_from_Modified
740,741,70,RM,60.0,9600,Pave,3,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,2Story,5,7,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,3,4,BrkTil,2.0,2.0,1.0,1.0,0,1.0,0,264,264,GasA,5,1,SBrkr,768,664,0,1432,0,0,2,0,2,1,3,7,Typ,0,Detchd,1910.0,1.0,2,360,3.0,4.0,2,270,0,112,0,0,0,0,5,WD,Abnorml,97,5
428,429,20,RL,64.0,6762,Pave,3,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,108.0,4,3,PConc,4.0,3.0,1.0,6.0,664,1.0,0,544,1208,GasA,5,1,SBrkr,1208,0,0,1208,1,0,2,0,2,1,4,6,Typ,0,Attchd,2007.0,2.0,2,628,3.0,3.0,2,105,54,0,0,0,0,0,9,New,Partial,0,0
664,665,20,RL,49.0,20896,Pave,3,Lvl,AllPub,CulDSac,Gtl,Somerst,RRAn,Norm,1Fam,1Story,8,5,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,3,PConc,5.0,3.0,2.0,6.0,1721,1.0,0,356,2077,GasA,5,1,SBrkr,2097,0,0,2097,1,0,1,1,1,1,5,8,Typ,1,Attchd,2005.0,3.0,3,1134,3.0,3.0,2,192,267,0,0,0,0,0,1,New,Partial,1,0
1321,1322,20,RL,60.0,6627,Pave,2,Lvl,AllPub,Corner,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,3,6,Hip,CompShg,VinylSd,VinylSd,None,0.0,3,3,CBlock,0.0,0.0,0.0,0.0,0,0.0,0,0,0,Floor,3,0,SBrkr,720,0,0,720,0,0,1,0,2,1,3,4,Typ,0,Detchd,1955.0,1.0,1,287,3.0,2.0,2,0,0,0,0,0,0,0,7,WD,Normal,59,58
654,655,20,RL,91.0,10437,Pave,2,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,1Story,8,6,Hip,CompShg,MetalSd,MetalSd,BrkFace,660.0,4,4,PConc,4.0,3.0,4.0,6.0,1696,1.0,0,413,2109,GasA,5,1,SBrkr,2113,0,0,2113,1,0,2,1,2,1,4,7,Typ,1,Attchd,1995.0,3.0,3,839,3.0,3.0,2,236,46,0,0,0,0,0,8,WD,Normal,13,13
1368,1369,120,RM,60.0,4435,Pave,3,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,TwnhsE,1Story,6,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,170.0,4,3,PConc,4.0,3.0,3.0,6.0,685,1.0,0,163,848,GasA,5,1,SBrkr,848,0,0,848,1,0,1,0,1,1,4,4,Typ,0,Attchd,2003.0,3.0,2,420,3.0,3.0,2,140,0,0,0,0,0,0,6,WD,Normal,6,5
1411,1412,50,RL,80.0,9600,Pave,3,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1.5Fin,6,8,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,4,CBlock,3.0,3.0,1.0,4.0,120,1.0,0,736,856,GasA,5,1,SBrkr,1112,556,0,1668,0,0,1,1,3,1,3,6,Min2,0,Attchd,1950.0,1.0,1,271,3.0,3.0,2,0,0,0,0,0,0,0,9,WD,Normal,59,4
584,585,50,RM,51.0,6120,Pave,3,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Fin,4,7,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,3,BrkTil,3.0,3.0,1.0,1.0,0,1.0,0,884,884,GasA,5,1,SBrkr,989,584,0,1573,0,0,1,0,3,1,4,6,Typ,0,Detchd,1935.0,1.0,1,240,3.0,3.0,2,0,0,54,0,120,0,0,7,WD,Normal,74,14
956,957,160,RM,24.0,1300,Pave,3,Lvl,AllPub,Inside,Gtl,Blueste,Norm,Norm,TwnhsE,2Story,6,6,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,CBlock,4.0,3.0,1.0,5.0,285,1.0,0,276,561,GasA,3,1,SBrkr,561,668,0,1229,0,0,1,1,2,1,3,5,Typ,1,Attchd,1980.0,3.0,2,462,3.0,3.0,2,150,0,0,0,0,0,0,5,WD,Normal,29,29
760,761,20,RL,70.0,9100,Pave,3,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,3,3,CBlock,3.0,3.0,1.0,3.0,612,1.0,0,252,864,GasA,5,1,SBrkr,864,0,0,864,0,0,1,0,2,1,3,5,Typ,0,Detchd,2008.0,1.0,1,300,5.0,5.0,2,0,0,0,0,0,0,450,10,WD,Normal,50,50


### 4. OneHotEncoding the categorical ordinal columns

In [13]:
#Removing already dropped columns from the OHE column scope
for i in set(categorical_ordinal_columns).intersection(drop_coulmns):
  categorical_ordinal_columns.remove(i)

In [14]:
from sklearn.preprocessing import OneHotEncoder

#Declaring the encoder
onehot_encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')
onehot_encoder=onehot_encoder.fit(df[categorical_ordinal_columns])

#Transforming the data
onehot_encoded = onehot_encoder.transform(df[categorical_ordinal_columns])
ohe_columns=onehot_encoder.get_feature_names_out(categorical_ordinal_columns)
df_ip_ohe=pd.DataFrame(onehot_encoded,columns=ohe_columns)

#Replacing the original data with OneHotEncoded data
df_ip.drop(categorical_ordinal_columns,axis=1,inplace=True)
df_ip= pd.merge(df_ip, df_ip_ohe, left_index=True, right_index=True)

print('Shape',df_ip.shape)
df_ip.sample(10)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Shape (1460, 222)


,Id,MSSubClass,LotFrontage,LotArea,LotShape,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,House_age_from_built,House_age_from_Modified,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGrg,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1315,1316,60,85.0,11075,3,6,5,0.0,3,3,2.0,3.0,2.0,5.0,500,2.0,276,176,952,3,1,1092,1020,0,2112,0,0,2,1,4,1,3,9,2,1969.0,1.0,2,576,3.0,3.0,2,280,0,0,0,0,0,0,6,39,39,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0

In [23]:
#Saving the preprecessed data for model building

FinalData=df_ip.copy()
FinalData['SalePrice']=df_op['SalePrice']

FinalData.to_csv('/content/drive/MyDrive/Projects/Kaggle/02. House Price Prediction/01. DataSet/01_Preprocessed_data.csv',index= False)